In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
from sklearn.model_selection import train_test_split
from shutil import copyfile

mainDataDir='/kaggle/input/multi-cancer/Multi Cancer/Brain Cancer'
brainGliomaDir=os.path.join(mainDataDir,'brain_glioma')
brainMeninDir=os.path.join(mainDataDir,'brain_menin')
brainTumorDir=os.path.join(mainDataDir,'brain_tumor')

In [4]:
trainDir = '/kaggle/working/training_data'
valDir = '/kaggle/working/validation_data'

In [5]:
os.makedirs(trainDir,exist_ok=True)
os.makedirs(valDir,exist_ok=True)

In [6]:
def split_and_copy(class_dir, train_output_dir, val_output_dir):
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.endswith(('jpg', 'jpeg', 'png'))]

    train_images, val_images = train_test_split(images, test_size=0.3, random_state=42)

    os.makedirs(train_output_dir, exist_ok=True)
    os.makedirs(val_output_dir, exist_ok=True)

    for img in train_images:
        copyfile(img, os.path.join(train_output_dir, os.path.basename(img)))

    for img in val_images:
        copyfile(img, os.path.join(val_output_dir, os.path.basename(img)))

split_and_copy(brainGliomaDir, os.path.join(trainDir, 'brain_glioma'), os.path.join(valDir, 'brain_glioma'))
split_and_copy(brainMeninDir, os.path.join(trainDir, 'brain_menin'), os.path.join(valDir, 'brain_menin'))
split_and_copy(brainTumorDir, os.path.join(trainDir, 'brain_tumor'), os.path.join(valDir, 'brain_tumor'))

## Data Augmentation

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
    '/kaggle/working/training_data',
    target_size=(224,224),
    class_mode='binary',
    batch_size=32
)

val_generator=val_datagen.flow_from_directory(
    '/kaggle/working/validation_data',
    target_size=(224,224),
    class_mode='binary',
    batch_size=32
)

Found 10500 images belonging to 3 classes.
Found 4500 images belonging to 3 classes.


## Training the Data

In [8]:
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Input,InputLayer,GlobalAveragePooling1D,BatchNormalization

In [9]:
numClasses=len(train_generator.class_indices)

In [10]:
numClasses

3

In [11]:
imageShape=(224,224,3)

In [67]:
from tensorflow.keras.regularizers import l2

model=Sequential([
    Input(shape=imageShape),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.2),
    Flatten(),
    Dense(128,activation='relu'),
    Dense(numClasses,activation='softmax')
])

In [68]:
from keras.optimizers import Adam

In [69]:
optimizer = Adam(learning_rate=0.001)  # Reduce learning rate
model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [70]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │     2,769,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,788,803 (10.64 MB)

 Trainable params: 2,788,803 (10.64 MB)

 Non-trainable params: 0 (0.00 B)

In [72]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
es=EarlyStopping( monitor="val_loss", patience=3,
                                     verbose=1,  restore_best_weights=True)
rlronp=ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=1,
                                             verbose=1)
callbacks=[es, rlronp]

In [74]:
batch_size=16
steps_per_epoch = len(train_generator) // batch_size  
validation_steps = len(val_generator) // batch_size  
history = model.fit(train_generator,validation_data=val_generator,epochs=100,steps_per_epoch=steps_per_epoch,validation_steps=validation_steps,callbacks=[earlystop])

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.4044 - loss: 1.0765 - val_accuracy: 0.4844 - val_loss: 0.9659
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.5002 - loss: 0.9628 - val_accuracy: 0.5820 - val_loss: 0.8060
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.5605 - loss: 0.9619 - val_accuracy: 0.6094 - val_loss: 0.7660
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.5461 - loss: 0.8886 - val_accuracy: 0.6875 - val_loss: 0.7008
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.6507 - loss: 0.8025 - val_accuracy: 0.6719 - val_loss: 0.7116
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.6279 - loss: 0.8126 - val_accuracy: 0.6719 - val_loss: 0.6987
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.6379 - loss: 0.8050 - val_accuracy: 0.7227 - val_loss: 0.6629
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.6345 - loss: 0.7882 - val_accuracy: 0.6992 - v

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 563ms/step - accuracy: 0.6764 - loss: 0.7415 - val_accuracy: 0.7656 - val_loss: 0.6002
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.7195 - loss: 0.6540 - val_accuracy: 0.6959 - val_loss: 0.6374
Epoch 19/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.6991 - loss: 0.7156 - val_accuracy: 0.7461 - val_loss: 0.5876
Epoch 20/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.6706 - loss: 0.7286 - val_accuracy: 0.7148 - val_loss: 0.6184
Epoch 21/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.6542 - loss: 0.7574 - val_accuracy: 0.7383 - val_loss: 0.6041
Epoch 22/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.6724 - loss: 0.7409 - val_accuracy: 0.7812 - val_loss: 0.5533
Epoch 23/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.7300 - loss: 0.6536 - val_accuracy: 0.7617 - val_loss: 0.5854
Epoch 24/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.7111 - loss: 0.7192 - val_accuracy: 0.7500 - val

In [75]:
# Evaluate on the training generator
train_results = model.evaluate(train_generator,batch_size=32)
print("Train loss, Train accuracy:", train_results)

# Evaluate on the validation generator
val_results = model.evaluate(val_generator)
print("Validation loss, Validation accuracy:", val_results)

329/329 ━━━━━━━━━━━━━━━━━━━━ 204s 619ms/step - accuracy: 0.7074 - loss: 0.7020
Train loss, Train accuracy: [0.6965799331665039, 0.7104762196540833]
141/141 ━━━━━━━━━━━━━━━━━━━━ 47s 334ms/step - accuracy: 0.7708 - loss: 0.6081
Validation loss, Validation accuracy: [0.6015225648880005, 0.7680000066757202]


In [76]:
def savingModel(model, model_name="model"): 
    model_filename = f"{model_name}.h5"
    model.save(model_filename)
    print(f"Model saved as {model_filename}")

In [82]:
savingModel(model,model_name="BrainCancer")

Model saved as BrainCancer.h5


In [83]:
from keras.models import load_model

In [84]:
def loadingModel(model_name="model"): 
    try:
        model = load_model(f"{model_name}.h5")
        model.summary()  # Print model summary if needed
        return model  # Return the loaded model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

In [85]:
loadedModel=loadingModel("BrainCancer")

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │     2,769,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,788,805 (10.64 MB)

 Trainable params: 2,788,803 (10.64 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [86]:
def loadImg(imgPath): 
    img = tf.keras.preprocessing.image.load_img(imgPath, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array=img_array/255.0
    return img_array

In [87]:
import tensorflow as tf

In [88]:
def predictImagesInFolder(testFolder, class_name, model_name): 
    savedModel = loadingModel(model_name)
    print(savedModel)

    if savedModel is None:
        print("Model loading failed.")
        return

    for subdir, dirs, files in os.walk(testFolder): 
        if subdir == testFolder: 
            continue
        trueLabel = os.path.basename(subdir)
        for file in files: 
            img_path = os.path.join(subdir, file)
            img = loadImg(img_path)
            
            # Debugging prediction probabilities
            predictions = savedModel.predict(img)
#             print(f"Prediction Probabilities for {file}: {predictions}")

            predicted_class_index = np.argmax(predictions, axis=1)
            predicted_class = class_name[predicted_class_index[0]]
            print(f"Predicted class: {predicted_class} File: {file}")

In [89]:
def testFolder(folder): 
    testFolder=f"{folder}"
    return testFolder

In [90]:
ALLCancerTestFolder=testFolder("/kaggle/working/validation_data")

In [91]:
train_generator.class_indices

{'brain_glioma': 0, 'brain_menin': 1, 'brain_tumor': 2}

In [92]:
ALLCancerClassname={0:'brain_glioma', 1:'brain_menin', 2:'brain_tumor'}


In [93]:
predictImagesInFolder(ALLCancerTestFolder,ALLCancerClassname,model_name="BrainCancer")

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │     2,769,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,788,805 (10.64 MB)

 Trainable params: 2,788,803 (10.64 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

<Sequential name=sequential_7, built=True>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Predicted class: brain_menin File: brain_menin_3721.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted class: brain_menin File: brain_menin_4079.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted class: brain_menin File: brain_menin_1963.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted class: brain_glioma File: brain_menin_1904.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted class: brain_menin File: brain_menin_2467.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: brain_menin File: brain_menin_2672.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: brain_menin File: brain_menin_3983.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: brain_menin File: brain_menin_4379.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted class: brain_tumor File: brain_menin_4882.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted class: brain_menin File: brain_menin_4475.jpg
1/1 ━━━━━━━━━━━

KeyboardInterrupt: 